In [158]:
import pymysql.cursors
from bs4 import BeautifulSoup
import requests


HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) ' + \
                         'Chrome/86.0.4240.111 Safari/537.36',
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;' + \
                     'q=0.8,application/signed-exchange;v=b3;q=0.9'}
HOST = 'https://www.b2b-center.ru'

MYSQL_HOST='localhost'
MYSQL_USER='proxy_user'
MYSQL_PWD='proxy_user12345'
MYSQL_DB='proxy_servers'

def get_html(url, proxies=None, timeout=None):
    r = requests.get(url, headers=HEADERS, proxies=proxies, timeout=timeout)
    return r

# Чтение из базы
def mysql_read(sql):
    connection = pymysql.connect(host=MYSQL_HOST,
                                 user=MYSQL_USER,
                                 password=MYSQL_PWD,
                                 db=MYSQL_DB,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            return cursor
    finally:
        connection.close()

# Запись в базу
def mysql_write(sql):
    connection = pymysql.connect(host=MYSQL_HOST,
                                 user=MYSQL_USER,
                                 password=MYSQL_PWD,
                                 db=MYSQL_DB,
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)

    try:
        with connection.cursor() as cursor:
            # Create a new record
            cursor.execute(sql)
        # connection is not autocommit by default. So you must commit to save
        # your changes.
        connection.commit()

    finally:
        connection.close()

def is_ip(ip):
    d = ip.split('.')
    if not len(d) == 4 or not all(map(str.isdigit, d)):
        return False
    return all(list(map(lambda x: int(x)<=255, d)))

def get_proxy_type(proxy): # Возвращает id типа прокси по текстовой строке
    proxy = proxy.lower().strip()
    cursor = mysql_read(f'SELECT id FROM proxy_type WHERE type = "{proxy}"')
    if cursor.rowcount == 0:
        return 1
    else:
        return cursor.fetchone()['id']

def get_proxy_from_scrapingant():
    START_URL = 'https://scrapingant.com/free-proxies/'
    html = get_html(START_URL, timeout=10)
    soup = BeautifulSoup(html.text, 'html.parser')
    search_table = soup.find('table', class_='proxies-table')
    trs = search_table.find_all('tr')
    values = []
    for tr in trs:
        td = tr.find_all('td')
        if len(td) == 0 or not is_ip(td[0].text):  # Если текст в ячейке не похож на ip
            continue
        else:
            # Если такого ip еще нет в базе, добавляем
            if mysql_read(f'SELECT 1 FROM proxy_servers WHERE ip = "{td[0].text}"').rowcount == 0:
                ip = td[0].text
                port = td[1].text
                p_type = get_proxy_type(td[2].text)
                values.append(f'({p_type}, "{ip}", {port})')
    add_query = 'INSERT INTO proxy_servers (proxy_type_id, ip, port) VALUES \n' + ',\n'.join(values) + ';'
    mysql_write(add_query)

# Возвращает присутствует ли указанный ip в качестве подстроки в списке 
def is_proxy_in(proxy_list, ip):
    return any(ip in p_string for p_string in proxy_list)

In [176]:
%%time
p_type = '3'
values = []
try:
    f = open('proxy_socks5.txt', 'r')
    proxyes = f.read()
finally:
    f.close()
p_list = proxyes.split('\n')
for p in p_list:
    ip = p.split(':')[0]
    port = p.split(':')[1]
    if mysql_read(f'SELECT 1 FROM proxy_servers WHERE ip = "{ip}"').rowcount == 0 \
                                                    and not is_proxy_in(values, ip):
        values.append(f'({p_type}, "{ip}", {port})')
add_query = 'INSERT INTO proxy_servers (proxy_type_id, ip, port) VALUES \n' + ',\n'.join(values) + ';'
mysql_write(add_query)

Wall time: 574 ms


In [177]:
get_proxy_from_scrapingant()